#### Este baseline se basara en armar un modelo LSTM por cada producto, con una optimizacion de hiper parametros escueta, para poder comparar con futuros experimientos. En caso de que esta alternativa funcione bien, seria recomendable incorporar parametros de optimizacion extra.

#### Imports

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [3]:
final_dataset = pd.read_csv('../../Datasets/final_dataset.csv', sep='\t')

In [4]:
final_dataset.head()

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age
0,201701,20001,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
1,201702,20001,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,1
2,201703,20001,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,1,2
3,201704,20001,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,3
4,201705,20001,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,4


In [5]:
columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter','y']

#### Funcion para crear los modelos

#### Armado de los modelos

In [10]:
import os
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
import joblib

product_ids = final_dataset['product_id'].unique()
predictions = []
    
for product_id in product_ids:
    product_data = final_dataset[final_dataset['product_id'] == product_id].sort_values(by='periodo')[columns]
    
    product_data_array = product_data['y'].values
    
    model = auto_arima(product_data_array, seasonal=True, trace=True, error_action='ignore', suppress_warnings=True)

    os.makedirs('Arima_results', exist_ok=True)
    joblib.dump(model, f'Models_params/model_product_{product_id}.pkl')
    
    forecast_steps = 2
    forecast = model.predict(n_periods=forecast_steps)
    
    # Agregar predicción al resultado
    predicted_y = forecast[-1]
    predictions.append({'product_id': product_id, 'predicted_y': predicted_y})

    print(f'Modelo para el producto {product_id} entrenado y guardado. Predicción a 2 meses: {predicted_y}')

 ARIMA(2,1,1)(0,0,0)[0]             : AIC=388.802, Time=0.32 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=386.839, Time=0.93 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=384.903, Time=0.46 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=386.835, Time=0.40 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=391.624, Time=0.05 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=388.832, Time=0.95 sec

Best model:  ARIMA(0,1,1)(0,0,0)[0]          
Total fit time: 11.741 seconds
Modelo para el producto 20050 entrenado y guardado. Predicción a 2 meses: 132.36502910408538
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=402.811, Time=1.59 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=405.454, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=405.028, Time=0.04 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=405.742, Time=0.42 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=479.128, Time=0.01 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=402.985, Time=0.90 sec
 ARIMA

In [ ]:
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv('../../Datasets/predictions.csv', index=False)

print('Todas las predicciones han sido generadas y guardadas en predictions.csv.')

Todas las predicciones han sido generadas y guardadas en predictions.csv.
